# Enhanced Web Game Controller: HCI Application

In this notebook, we analyze and modify an existing Human-Computer Interaction (HCI) application based on a specific use case. We identify usability issues in the original application and implement enhancements to improve user experience and accessibility.

## Overview

This notebook covers:
1. Analysis of usability issues in the original application
2. Proposed improvements to enhance usability
3. Implementation of these improvements
4. Explanation of how these modifications impact user experience

## 1. Analysis of Usability Issues

After reviewing the original code (`06_02_Web_game.py`), we identified several usability issues:

1. **Fixed Bounding Box Size**: The control rectangle is a fixed size (300x400 pixels) and doesn't adapt to different users or camera setups.
2. **Binary Input System**: Controls are binary (press or don't press), with no option for gradual or proportional movement.
3. **Single Input Method**: Only supports face detection, limiting accessibility for users with mobility challenges.
4. **Limited Visual Feedback**: The user can see their face and the center box, but there's no clear indication of which direction is being activated.
5. **No Calibration Option**: Users can't adjust sensitivity or control box size to match their comfort level.
6. **No Pause Functionality**: Users can't pause the game without exiting completely (Esc key).
7. **Fixed Click Position**: The game always clicks at position (500, 500) to start, which might not align with different web games.
8. **Minimal Error Handling**: Limited error handling for camera initialization or face detection failures.

## 2. Proposed Improvements

Based on these issues, we implemented the following improvements:

1. **Adjustable Control Box**: Allow users to resize the control area during gameplay.
2. **Enhanced Visual Feedback**: Add directional indicators to show which controls are being activated.
3. **Alternative Input Methods**: Add hand gesture control as an alternative to face detection.
4. **Pause Functionality**: Implement a pause feature with a simple gesture.
5. **Calibration System**: Add calibration for different users and environments.
6. **Customizable Click Position**: Allow the user to specify the game's start position.
7. **Improved Error Handling**: Better feedback for camera or detection issues.

In [1]:
# Import necessary libraries
import cv2
import numpy as np
import pyautogui as gui
import time
import argparse

# Set keypress delay to 0
gui.PAUSE = 0

## 3. Implementation

The enhanced implementation is available in the `Enhanced_Web_game.py` file. Here we'll walk through the key improvements.

### 3.1 Alternative Input Method: Hand Detection

We added hand gesture control as an alternative to face detection for users who may have difficulty with head movements or prefer hand controls.

In [2]:
def detect_hand(frame):
    '''
    Detect hands in the frame using MediaPipe or color-based detection as fallback.
    
    returns: Hand position (x, y) or None if no hand detected
    '''
    # This is a simplified version - see the full implementation in the .py file
    # Code attempts to use MediaPipe if available, otherwise falls back to color-based detection
    
    # Simple color-based hand detection (example)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # Find contours and return largest (simplified)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > 5000:
        # Return centroid of largest contour
        # (implementation details in .py file)
        return (cx, cy)
    
    return None

### 3.2 Enhanced Visual Feedback

We added directional indicators and improved UI elements to provide better feedback to the user.

In [3]:
def draw_ui_elements(frame, bbox, fps):
    '''
    Draw UI elements on the frame for better user feedback
    '''
    # Draw control rectangle
    frame = cv2.rectangle(frame, (left_x, top_y), (right_x, bottom_y), (0, 0, 255), 2)
        
    # Draw directional indicators when out of center
    if last_mov == 'left':
        cv2.arrowedLine(frame, (center_x, center_y), (center_x - 50, center_y), (255, 0, 0), 3)
    elif last_mov == 'right':
        cv2.arrowedLine(frame, (center_x, center_y), (center_x + 50, center_y), (255, 0, 0), 3)
    # Similar code for up/down directions
    
    # Show status information (FPS, control mode, etc.)
    cv2.putText(frame, f"FPS: {fps}", (frame_width - 150, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Show current control mode and other UI elements
    # (implementation details in .py file)
    
    return frame

### 3.3 Calibration System

We implemented a calibration mode that allows users to adjust the control box size and sensitivity.

In [4]:
# Key handling in main loop (simplified)
'''
k = cv2.waitKey(5)
if k == 27:  # Esc key
    break
elif k == ord('p') or k == ord('P'):  # Pause/Resume
    is_paused = not is_paused
elif k == ord('c') or k == ord('C'):  # Calibration mode
    calibration_mode = not calibration_mode
elif k == ord('+') or k == ord('='):  # Increase box size
    if calibration_mode:
        bbox_size = min(bbox_size + 10, frame_width // 2 - 20)
        bbox_height = min(bbox_height + 10, frame_height // 2 - 20)
elif k == ord('-') or k == ord('_'):  # Decrease box size
    if calibration_mode:
        bbox_size = max(bbox_size - 10, 50)
        bbox_height = max(bbox_height - 10, 50)
elif k == ord('s') or k == ord('S'):  # Adjust sensitivity
    sensitivity = (sensitivity + 0.25) % 2.0
    if sensitivity == 0:
        sensitivity = 0.25
'''

"\nk = cv2.waitKey(5)\nif k == 27:  # Esc key\n    break\nelif k == ord('p') or k == ord('P'):  # Pause/Resume\n    is_paused = not is_paused\nelif k == ord('c') or k == ord('C'):  # Calibration mode\n    calibration_mode = not calibration_mode\nelif k == ord('+') or k == ord('='):  # Increase box size\n    if calibration_mode:\n        bbox_size = min(bbox_size + 10, frame_width // 2 - 20)\n        bbox_height = min(bbox_height + 10, frame_height // 2 - 20)\nelif k == ord('-') or k == ord('_'):  # Decrease box size\n    if calibration_mode:\n        bbox_size = max(bbox_size - 10, 50)\n        bbox_height = max(bbox_height - 10, 50)\nelif k == ord('s') or k == ord('S'):  # Adjust sensitivity\n    sensitivity = (sensitivity + 0.25) % 2.0\n    if sensitivity == 0:\n        sensitivity = 0.25\n"

### 3.4 Improved Movement Control

We enhanced the movement function to provide proportional control based on the distance from the center.

In [5]:
# Proportional control snippet (simplified)
'''
# Distance from center (for proportional control)
x_distance = abs(face_center_x - center_x) / center_x
y_distance = abs(face_center_y - center_y) / center_y

# Scale by sensitivity
x_distance *= sensitivity
y_distance *= sensitivity

# Apply controls based on position with proportional response
if x1 < bbox[0]:
    # Repeat key presses based on distance for smoother control
    presses = max(1, int(x_distance * 3))
    for _ in range(presses):
        gui.press('left')
'''

"\n# Distance from center (for proportional control)\nx_distance = abs(face_center_x - center_x) / center_x\ny_distance = abs(face_center_y - center_y) / center_y\n\n# Scale by sensitivity\nx_distance *= sensitivity\ny_distance *= sensitivity\n\n# Apply controls based on position with proportional response\nif x1 < bbox[0]:\n    # Repeat key presses based on distance for smoother control\n    presses = max(1, int(x_distance * 3))\n    for _ in range(presses):\n        gui.press('left')\n"

## 4. Impact on User Experience

The enhancements we implemented significantly improve the user experience in several ways:

1. **Improved Accessibility**: The addition of hand gesture control provides an alternative for users who may have difficulty with head movements.

2. **Enhanced Feedback**: Directional indicators and UI elements provide clear feedback about the system state, helping users understand what's happening.

3. **Customization**: The calibration system allows users to adjust the control box size and sensitivity to match their preferences and environment.

4. **Smoother Control**: Proportional control based on distance from center provides more nuanced and responsive gameplay than binary controls.

5. **Better User Control**: Pause functionality and error handling give users more control over the application.

These improvements make the application more usable, accessible, and enjoyable for a wider range of users.

## 5. Running the Enhanced Application

To run the enhanced application, execute the following command in your terminal:

```
python Enhanced_Web_game.py
```

You can also provide command-line arguments to customize the behavior:

```
python Enhanced_Web_game.py --mode hand --click-pos 600 400 --sensitivity 1.25 --box-size 200 --box-height 250
```

While the application is running, you can use the following keyboard controls:
- P: Pause/Resume the game
- C: Enter calibration mode (adjust control box size)
- M: Switch between face and hand control modes
- H: Show/hide help text
- S: Adjust sensitivity
- +/-: Increase/decrease box size (in calibration mode)
- Esc: Exit

## 6. Conclusion

Through this assignment, we've learned how to analyze an HCI application, identify usability issues, and implement improvements to enhance the user experience. The modified application demonstrates principles of good HCI design, including:

- **Accessibility**: Supporting different input methods for diverse users
- **Feedback**: Providing clear visual cues about system state
- **User Control**: Allowing customization and providing responsive controls
- **Error Handling**: Gracefully handling potential errors and providing feedback

These enhancements make the application more usable and accessible, demonstrating the importance of considering human factors in interface design.